<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/TIC/Aula_06_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>


In [ ]:
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns

import warnings
warnings.filterwarnings("ignore")

# **Modelo K-Nearest Neighbor (KNN)**

Assim como a Regressão Logística, O algoritmo dos K-Vizinhos Mais Próximos (KNN) é um algoritmo de aprendizado de máquina usado para classificação. O KNN classifica cada valor de um conjunto de dados avaliando sua distância em relação aos k vizinhos mais próximos. Se os k vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria. A ideia é a seguinte:

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_knn_ex1.png'>

Quando o k é pequeno, a classificação fica mais sensível a regiões bem próximas (podendo ocorrer o problema de *overfitting*). Com k grande, a classificação fica menos sujeita a ruídos e pode ser considerada mais robusta. Porém, se k for grande demais, pode ser que haja o problema de *underfitting*.

Lembre-se:
- *Overfitting*: O modelo tem um ótimo desempenho nos dados de treinamento, mas o resultado nos dados de teste é ruim. O modelo se especializou nos dados de treinamento (não tem capacidade de generalização).

- *Underfitting*: O desempenho do modelo já é ruim no próprio treinamento. Na etapa de treinamento, não houve o aprendizado de relações entre as variáveis e resultado pode ser visto como algo aleatório.

Em nosso exemplo, veja o que acontece se consideramos 5 vizinhos próximos.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_knn_ex2.png'>

<font size =5 > Algoritmo KNN </font>

1.   Selecione o dado a ser classificado;
1.   Meça sua distância (euclidiana) em relação a cada um dos outros dados que já estão classificados (os dados de treinamento);
1.   Selecione os primeros k dados com menores distâncias;
1.   Verifique as classes (ou categorias) dos dados que tiveram as k menores distâncias e conte a quantidade de vezes que cada classe apareceu;
1.   Classifique esse novo dado como pertencente à classe que mais apareceu.

<font size =4 > Funções Distância </font>

Existem várias funções distância, mas em geral aplicaremos aqui a distância Euclidiana:

$$ d(\hat{x},x^i) = \sqrt{ \sum_{j=1}^n(\hat{x}_j - x_j^i)^2 },$$

onde $\hat{x} = [x_1, x_2, \ldots, x_j]$ é o dado a ser classificado e $x^i = [x_1^i, x_2^i, \ldots, x_j^i]$ é um dado de treinamento. Esse cálculo, então, é feito para todos os $n$ dados de teinamento $i = 1, \ldots, n$.






### Prós
* É simples de implementar;
* Treinar é mais fácil;
* Tem poucos parâmetros.

### Limitações
* Alto custo de previsão;
* Requer o uso de uma função distância.



## Exemplo
Utilizaremos um exemplo bastante simples com o objetivo de facilitar a compreensão do modelo. Neste exemplo, a classificação 'O cliente vai pagar o empréstimo?', que pode ser Sim (o empréstimo foi concedido e pago pelo cliente - classe 1) ou Não (o empréstimo foi concedido e não foi pago pelo cliente - classe 0), é determinada com base em apenas dois atributos: a idade do cliente e o valor do empréstimo.

Note que, para essa construir esse classificador, necessitamos de um conjunto de dados que indique clientes que já pagaram ou não um empréstimo obtido.

In [ ]:
Loans = pd.DataFrame({'Age':[40,35,45,34,45,48,53,60,50,48,53],
                      'Loan':[100000,60000,80000,40000,140000,120000,95000,92000,100000,152000,150000],
                      'Repayment':[1,1,1,1,1,1,0,0,0,0,0] }) # 1=vai pagar 0=não vai pagar

Loans

Qual deve ser a classificação (vai pagar o não o empréstimo) para um cliente com 47 anos e um emprestimo de 142.000? Vamos visualizar os dados:

In [ ]:
# Visualizando os dados!

sns.scatterplot(data=Loans, x='Age', y='Loan', hue='Repayment')
plt.scatter(47, 142000, c='fuchsia',s=60) # Novo valor a ser classificado
plt.show()

In [ ]:
# Calculando as distâncias

dist = []
for i in range(len(Loans)):
    d = float( np.sqrt( (Loans.iloc[i].Age - 47)**2 + (Loans.iloc[i].Loan - 142000)**2 ) )
    dist.append(d)

Loans['dist'] = dist

Loans.sort_values('dist')

Considerando k=3 (os três vizinhos mais próximos) a classificação para esse novo cliente seria: Não vai pagar o empréstimo (*Repayment* = 0).


<font size =5> **Cuidado!** Normalizar os dados </font>

A distância euclidiana é sensível à escala dos dados, o que significa que variáveis com escalas diferentes podem afetar o resultado. Para evitar esse problema, é comum normalizar ou padronizar os dados antes de calcular a distância euclidiana. Isso garante que todas as variáveis tenham a mesma influência na medida de distância.

Portanto, antes de calcular distâncias euclidianas em conjuntos de dados com variáveis de escalas diferentes, é recomendável normalizar ou padronizar os dados para obter resultados mais robustos e significativos.

Há diversas formas de normalizar dados. Aqui, vamos considerar a normalização pelo máximo. Ou seja, para cada atributo (coluna), dividimos os valores pelo máximo dessa mesma coluna. Isso garante que os dados fiquem entre 0 e 1.

In [ ]:
# Normalizando o novo dado

print("idade:", 47/Loans.Age.max())
print("Loan:", 142000/Loans.Loan.max())

In [ ]:
# Normalizando os dados

Loans["Age"] = Loans.Age / Loans.Age.max()
Loans["Loan"] = Loans.Loan / Loans.Loan.max()

In [ ]:
# Calculando as distâncias

dist = []
for i in range(11):
    d = float( np.sqrt( (Loans.iloc[i].Age - 0.7833)**2 + (Loans.iloc[i].Loan - 0.9342)**2 ) )
    dist.append(d)

Loans['dist'] = dist

Loans.sort_values('dist')

Considerando k=3 (os três vizinhos mais próximos) a classificação para esse novo cliente seria: Não vai pagar o empréstimo (*Repayment* = 0).

## Usando o módulo `neighbors` da biblioteca `sklearn`

Vamos agora usar o módulo `neighbors` da biblioteca `sklearn` para implementar o KNN. Também, neste exemplo, vamos considerar uma outra técnica de normalização dos dados. Neste caso, para cada dado de uma mesma coluna, subtraímos a média e dividimos pelo desvio padrão (desta mesma coluna). Como resultados, cada coluna terá média igual a 0 e desvio padrão igual a 1. Essa técnica também é chamada de padronização.

In [ ]:
from sklearn import neighbors

In [ ]:
Loans = pd.DataFrame({'Age':[40,35,45,34,45,48,53,60,50,48,53],
                      'Loan':[100000,60000,80000,40000,140000,120000,95000,92000,100000,152000,150000],
                      'Repayment':[1,1,1,1,1,1,0,0,0,0,0] }) # 1=vai pagar 0=não vai pagar

Loans.head(3)

In [ ]:
new_case=pd.DataFrame({'Age':[47],
                      'Loan':[142000]}) # 1=vai pagar 0=não vai pagar
new_case

In [ ]:
# Dados de entrada/saída
X = Loans[['Age','Loan']]
y = Loans[["Repayment"]]

# padroniza os dados para que a média seja 0 e o desvio padrão seja 1.
X_padronizados = (X-X.mean())/X.std()

# Definindo ou declarando o modelo
k=3
clf = neighbors.KNeighborsClassifier(k)

# Aprendizado (Emprega o conjunto de treinamento)
clf.fit(X_padronizados, y)

# Fazendo a predição
new_case_padronizado=(new_case-X.mean())/X.std()
Loan_type_pred = clf.predict(new_case_padronizado)

print('Classificação KNN, k=', k ,', para o novo empréstimo é ', Loan_type_pred)

<font size= 5> Adicionando variáveis categóricas </font>

Vamos agora incluir variáveis categóricas no problema de classificação. Nesse caso, vamos incluir uma variável que indica se o empréstimo é de curto prazo (*Short*), longo prazo (*Long*), ou prazo indefinido (*Undefined*).

In [ ]:
Loans = pd.DataFrame({'Age':[40,35,45,34,45,48,53,60,50,48,53],
                      'Loan':[100000,60000,80000,40000,120000,98000,95000,92000,100000,170000,150000],
                      'Duration':['Short','Long','Short','Undefined','Long','Short','Long','Short','Undefined','Long', 'Short'],
                      'Repayment':[1,1,1,1,1,1,0,0,0,0,0] }) # 1=vai pagar 0=não vai pagar
Loans.head()

In [ ]:
new_case  = pd.DataFrame({'Age':[47],
                      'Duration':['Short'],
                      'Loan':[142000]})
new_case

Uma forma de lidar com variáveis categóricas é de "binarizar" as categorias, de maneira a criar novas colunas de dados que contenha 1 quando a categoria está presente na amostra e 0 quando não está. Nesse caso, um atributo categórico, com $m$ categorias, é substituído por $m$ novas colunas.

Veja o exemplo a seguir.

In [ ]:
# Variáveis dummy (Hot encode)

dummies = pd.get_dummies(Loans["Duration"])
Loans = pd.concat([Loans, dummies],axis=1)
Loans.head()

In [ ]:
# Eliminando a coluna original
Loans.drop(columns=['Duration'], inplace=True)
Loans

In [ ]:
# New case

dummies_new_case = pd.get_dummies(new_case["Duration"])
new_case = pd.concat([new_case, dummies_new_case],axis=1)
new_case.drop(columns=['Duration'], inplace=True)
new_case.head()

In [ ]:
# Adicionando as colunas que faltam
new_case['Long'] = 0
new_case['Undefined'] = 0
new_case

In [ ]:
# Dados de entrada/saída
X = Loans[['Age','Loan']]
y = Loans[["Repayment"]]

# Normalizando os dados
X_padronizados = (X-X.mean())/X.std()
X_padronizados = pd.concat([X, dummies],axis=1) #adicionando dummies

# Definindo ou declarando o modelo
k=3
clf = neighbors.KNeighborsClassifier(k)

# Aprendizado (Emprega o conjunto de treinamento)
clf.fit(X_padronizados, y)

# Fazendo a predição
X_new_case=new_case[['Age','Loan']]
X_new_case_padr=(X_new_case-X.mean())/X.std()

X_new_case_padr=pd.concat([X_new_case_padr, new_case[["Long","Short","Undefined"]]],axis=1)

Loan_type_pred = clf.predict(X_new_case_padr)

print('Classificação Knn, k=', k ,', para o novo empréstimo é', Loan_type_pred)